In [1]:
import requests
import time
import csv

# Define constants
API_KEY = 'YOUR_ETHERSCAN_API_KEY'  # Replace with your Etherscan API key
WALLET_ADDRESS = '0xd4bC53434C5e12cb41381A556c3c47e1a86e80E3'
ETHERSCAN_API_URL = "https://api.etherscan.io/api"
START_BLOCK = 0  # Starting block (could be set to a higher value to narrow down the search)
END_BLOCK = 99999999  # Use a high block number for the latest block

OFFSET_LENGTH = 2500

def get_internal_transactions(wallet_address, start_block=START_BLOCK, end_block=END_BLOCK):
    transactions = []
    page = 1  # Start with page 1
    while True:
        # Call Etherscan API to get internal transactions
        params = {
            'module': 'account',
            'action': 'txlistinternal',
            'address': wallet_address,
            'startblock': start_block,
            'endblock': end_block,
            'page': page,
            'offset': OFFSET_LENGTH,  # Number of transactions per page
            'sort': 'desc',  # Sort by ascending block number
            'apikey': API_KEY
        }

        response = requests.get(ETHERSCAN_API_URL, params=params)
        data = response.json()

        if data['status'] == '1' and 'result' in data:
            transactions.extend(data['result'])
            print(len(transactions))

            # If there are more results, move to the next page
            if len(data['result']) == OFFSET_LENGTH:
                page += 1
            else:
                break  # All results retrieved
        else:
            print(f"Error: {data.get('message', 'Unknown error')}")
            break

        # Respect rate limits (about 5 requests per second)
        time.sleep(1)

    return transactions

def save_transactions_to_csv(transactions, filename='internal_transactions.csv'):
    # Define the CSV header
    header = ['BlockNumber', 'TimeStamp', 'Hash', 'From', 'To', 'Value', 'Gas', 'GasUsed', 'IsError', 'TxReceiptStatus']
    
    # Open CSV file for writing
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()  # Write the header row
        for transaction in transactions:
            writer.writerow({
                'BlockNumber': transaction.get('blockNumber'),
                'TimeStamp': transaction.get('timeStamp'),
                'Hash': transaction.get('hash'),
                'From': transaction.get('from'),
                'To': transaction.get('to'),
                'Value': transaction.get('value'),
                'Gas': transaction.get('gas'),
                'GasUsed': transaction.get('gasUsed'),
                'IsError': transaction.get('isError'),
                'TxReceiptStatus': transaction.get('txReceiptStatus')
            })

    print(f"Data saved to {filename}")

# Fetch internal transactions
transactions = get_internal_transactions(WALLET_ADDRESS)

# Print the results and save them to a CSV file
if transactions:
    print(f"Found {len(transactions)} internal transactions for wallet {WALLET_ADDRESS}")
    save_transactions_to_csv(transactions)
else:
    print("No internal transactions found.")

2500
5000
7500
10000
Error: Result window is too large, PageNo x Offset size must be less than or equal to 10000
Found 10000 internal transactions for wallet 0xd4bC53434C5e12cb41381A556c3c47e1a86e80E3
Data saved to internal_transactions.csv


In [2]:
import pandas as pd

# Load the CSV file containing the ERC-20 token transfers
file_path = 'internal_transactions.csv'  # Make sure the path is correct if not in the same directory
df_internal = pd.read_csv(file_path, index_col='Hash')

# Convert 'TimeStamp' from Unix timestamp to yyyy-mm-dd hh:mm:ss format
df_internal['TimeStamp'] = pd.to_datetime(df_internal['TimeStamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M')

# Rename 'Value' to 'Value_ETH' to represent Ether values
df_internal.rename(columns={'Value': 'Value_Eth'}, inplace=True)
df_internal.rename(columns={'GasPrice': 'GasPrice_Gwei'}, inplace=True)

# Convert 'Value_ETH' from Wei to Ether 
df_internal['Value_Eth'] = pd.to_numeric(df_internal['Value_Eth'], errors='coerce') / 10**18  # Convert Wei to Ether

# Save the modified DataFrame to the same CSV file
df_internal.to_csv('./csv/internal_transactions.csv')

In [3]:
df_internal

,BlockNumber,TimeStamp,From,To,Value_Eth,Gas,GasUsed,IsError,TxReceiptStatus
Hash,,,,,,,,,
0x021eb89b34ff1ba33e83afb1e69d619e30659ef21a8653f72e2c8bb1149ed348,21457799,2024-12-22 11:40,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12.774991,236027,4874,0,NaN
0x021eb89b34ff1ba33e83afb1e69d619e30659ef21a8653f72e2c8bb1149ed348,21457799,2024-12-22 11:40,0x52aa899454998be5b000ad077a46bbe360f4e497,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,12.774991,22300,513,0,NaN
0x68a0026d453a259f9fc0481469f3c5e406d9be955df50191882dd8bcb9cea551,21457757,2024-12-22 11:31,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0x836951eb21f3df98273517b7249dceff270d34bf,8.613897,655564,102817,0,NaN
0x68a0026d453a259f9fc0481469f3c5e406d9be955df50191882dd8bcb9cea551,21457757,2024-12-22 11:31,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,8.613897,2300,513,0,NaN
0x0bf7fde4163e0c40c3be5dbce5fb5930ea66f0acca3a370d111e5059fa9d6e1e,21457726,2024-12-22 11:25,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0x00000011f84b9aa48e5f8aa8b9897600006289be,0.009970,562243,86104,0,NaN
...,...,...,...,...,...,...,...,...,...
0x4887a9c8bdca4836cb1f4ca6abd49fda1e542782e4dc195218a3597368cde6d1,21370600,2024-12-10 07:25,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0.024034,2300,513,0,NaN
0x42958aca76b8444ccb3d0931cdb952b8ed1aa5c466a548591f0dcaaa3afc9b4b,21370555,2024-12-10 07:16,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0x00000011f84b9aa48e5f8aa8b9897600006289be,0.051554,223650,65765,0,NaN
0x42958aca76b8444ccb3d0931cdb952b8ed1aa5c466a548591f0dcaaa3afc9b4b,21370555,2024-12-10 07:16,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd4bc53434c5e12cb41381a556c3c47e1a86e80e3,0.051554,2300,513,0,NaN
